## ⚙️ Actualización Recaudo PSE
## Desarrollado por David Paez Ingeniero de datos
## Fecha de actualizacion 19/06/2025

In [ ]:
# --- Lista de festivos (2025 y 2026) ---
FESTIVOS = {
    # 2025
    date(2025, 1, 1), date(2025, 1, 6), date(2025, 3, 24), date(2025, 4, 17), date(2025, 4, 18),
    date(2025, 5, 1), date(2025, 6, 2), date(2025, 6, 23), date(2025, 6, 30), date(2025, 7, 20),
    date(2025, 8, 7), date(2025, 8, 18), date(2025, 10, 13), date(2025, 11, 3), date(2025, 11, 17),
    date(2025, 12, 8), date(2025, 12, 25),
    # 2026
    date(2026, 1, 1), date(2026, 1, 12), date(2026, 3, 23), date(2026, 4, 2), date(2026, 4, 3),
    date(2026, 5, 1), date(2026, 5, 18), date(2026, 6, 8), date(2026, 6, 15), date(2026, 6, 29),
    date(2026, 7, 20), date(2026, 8, 7), date(2026, 8, 17), date(2026, 10, 12), date(2026, 11, 2),
    date(2026, 11, 16), date(2026, 12, 8), date(2026, 12, 25),
}

# --- Función para obtener el primer día hábil del mes ---
def primer_dia_habil_del_mes(hoy: date, festivos: set) -> date:
    primer = date(hoy.year, hoy.month, 1)
    while primer.weekday() >= 5 or primer in festivos:
        primer += timedelta(days=1)
    return primer

# --- Evaluar si hoy es ese primer día hábil ---
hoy = date.today()
primer_habil = primer_dia_habil_del_mes(hoy, FESTIVOS)

if hoy == primer_habil:
    print(f"✅ Hoy ({hoy}) es el primer día hábil del mes. Ejecutando proceso...")
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    ## Extraccion
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    # ========================
    # #  CALCULO DE FECHAS SQL PARA EXTRAER EL PRIMER DIA DEL MES ANTERIOR Y ACTUAL
    # ========================
    inicio_mes_actual = hoy.replace(day=1)
    inicio_mes_anterior = (inicio_mes_actual - timedelta(days=1)).replace(day=1)
    fecha_inicio = inicio_mes_anterior.strftime('%Y-%m-%d')
    fecha_fin = inicio_mes_actual.strftime('%Y-%m-%d')

    # Conexión a df1
    server = 'fabogsqlclu,1433'
    database = 'GestionCliente'
    driver = '{ODBC Driver 17 for SQL Server}'
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

    # Consulta SQL con parámetros
    query = """
    SELECT C.Fechapago, C.HORPAG, C.MINPAG, C.TIPO_PRODUCTO,
           C.CFLNME AS Tipo_subproducto, C.CUX1AP, C.NUMDOC,
           C.CODSUC, C.NUMCRE, C.NOMCLI AS Nombre_Cliente,
           C.NUCREX, C.ORIPAG AS Origen_pago,
           CASE 
               WHEN ORIPAG = 1 THEN 'Conavi'
               WHEN ORIPAG = 2 THEN 'Banco caja social'
               WHEN ORIPAG = 3 THEN 'Banco de Bogotá'
               WHEN ORIPAG = 4 THEN 'Citibank'
               WHEN ORIPAG = 5 THEN 'Banco de occidente'
               WHEN ORIPAG = 6 THEN 'PSE'
               WHEN ORIPAG = 7 THEN 'Finazauto'
               WHEN ORIPAG = 8 THEN 'Bancolombia'
               WHEN ORIPAG = 10 THEN 'Banco de occidente ACH'
               WHEN ORIPAG = 11 THEN 'Banco AV Villas'
               WHEN ORIPAG = 12 THEN 'Cenit'
               WHEN ORIPAG = 13 THEN 'Banco Davivienda'
               WHEN ORIPAG = 14 THEN 'Cpv - Avvillas'
               WHEN ORIPAG = 15 THEN 'Libranza - Davivienda'
               WHEN ORIPAG = 16 THEN 'Sudameris'
               WHEN ORIPAG = 17 THEN 'Débito Automático'
               WHEN ORIPAG = 19 THEN 'Ws Bancolombia'
               WHEN ORIPAG IN (30, 31, 32) THEN 'Cajas Depositarias'
               WHEN ORIPAG = 63 THEN 'Cajas Finandina (Cheque TDC)'
               WHEN ORIPAG = 64 THEN 'Banca Línea - Finandina (TDC)'
               WHEN ORIPAG = 65 THEN 'Banco Finandina Pago TDC Dirigido'
               WHEN ORIPAG = 66 THEN 'Banco Finandina Pago Alterno TDC'
               ELSE NULL 
           END AS Descripcion_origen_pago,
           C.FORPAG AS Forma_pago, C.VLRPAG AS Valor_pagado,
           C.CIUREC, C.DESOBS, C.CODAPL
    FROM (
        SELECT CONCAT(SUBSTRING(B.FECPAG1,1,4), '-', SUBSTRING(B.FECPAG1,5,2), '-', SUBSTRING(B.FECPAG1,7,2)) AS Fechapago, B.*
        FROM (
            SELECT 
                CASE 
                    WHEN a.CUX1AP IN (50, 51, 53, 33, 30) THEN 'CREDITO'
                    WHEN a.CUX1AP = 20 THEN 'AHORRO'
                    WHEN a.CUX1AP IS NULL AND LEN(a.NUCREX) IN (5, 6, 15, 16) THEN 'TDC'
                    WHEN a.CUX1AP IS NULL AND LEFT(a.NUMCRE,1) = '4' THEN 'TDC'
                    WHEN a.CUX1AP IS NULL AND LEN(a.NUCREX) = 10 THEN 'CREDITO'
                    WHEN LEN(a.NUMCRE) = 16 AND LEFT(a.NUMCRE,1) = '4' THEN 'TDC'
                    WHEN a.CUX1AP IS NULL AND LEN(a.NUMCRE) = 16 THEN 'CREDITO'
                    ELSE 'NULL'
                END AS TIPO_PRODUCTO,
                CAST(FECPAG AS varchar) AS FECPAG1,
                A.*
            FROM OPENQUERY(DB2400_182, 
                'SELECT IFNULL(c.numdoc, cc.CUSSNR) AS numdoc,
                        CXP.CUX1AP,
                        pcl.*,
                        TP.CFLNME
                 FROM bnkprd01.pagcncl pcl
                 LEFT JOIN inttarcre.sattarjet t ON t.pan = pcl.nucrex
                 LEFT JOIN inttarcre.SATBENEFI b ON b.cuenta = t.cuenta
                 LEFT JOIN inttarcre.SATDACOPE c ON c.identcli = b.identcli
                 LEFT JOIN BNKPRD01.LNP00301 CR ON CR.LNNOTE = pcl.nucrex
                 LEFT JOIN BNKPRD01.CFP503 TP ON TP.CFTYP = CR.LNTYPE
                 LEFT JOIN BNKPRD01.CUP009 CXP ON CR.LNNOTE = CXP.CUX1AC
                 LEFT JOIN BNKPRD01.CUP00301 Cc ON Cc.CUNBR = CXP.CUX1CS
                 WHERE pcl.fecpag >= ''20250501'' AND pcl.fecpag < ''20250601'''
            ) A
        ) B
    ) C
    WHERE C.Fechapago >= ? AND C.Fechapago < ?
    """

    # Conexión y ejecución
    conn = pyodbc.connect(conn_str)
    df1 = pd.read_sql(query, conn, params=(fecha_inicio, fecha_fin))
    conn.close()
    # Conexion a df2
    #df2 es el mismo df5
    server = 'fabogriesgo\riesgodb,1433'  # Nota: doble barra y puerto separado por coma
    database = 'Productos y transaccionalidad'
    driver = '{ODBC Driver 17 for SQL Server}'
    # Crear cadena de conexión
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    # Conectar a la base de datos 
    conn = pyodbc.connect(conn_str)
    # Consulta SQL ProductoActivo
    query1 = """SELECT * FROM [Productos y transaccionalidad].[dbo].[ProductoActivo]"""
    # Ejecutar consulta y guardar resultado en DataFrame
    df2 = pd.read_sql(query1, conn)
    # Cerrar la conexión 
    conn.close()
    # Conexion a df3
    server = 'fabogsqlclu,1433'  # Nota: doble barra y puerto separado por coma
    database = 'PSE'
    driver = '{ODBC Driver 17 for SQL Server}'
    # Crear cadena de conexión
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    # Conectar a la base de datos 
    conn = pyodbc.connect(conn_str)
    # Consulta SQL ProductoActivo
    query2 = """SELECT * FROM fabogsqlclu.[PSE].[dbo].[TransactionsInfo]"""
    # Ejecutar consulta y guardar resultado en DataFrame
    df3 = pd.read_sql(query2, conn)
    # Cerrar la conexión 
    conn.close()
    
    
    # Conexion a df4
    server = 'fabogsqlclu,1433'  # Nota: doble barra y puerto separado por coma
    database = 'PSE'
    driver = '{ODBC Driver 17 for SQL Server}'
    # Crear cadena de conexión
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    # Conectar a la base de datos 
    conn = pyodbc.connect(conn_str)
    # Consulta SQL ProductoActivo
    query3 = """Select * ,CASE WHEN LEN(DBNROPRD) >= 11 THEN 'TDC' ELSE 'CREDITO' END AS 'TIPO_PRODUCTO' 
    from openquery (DB2400_182,'select * from INTERFACES.DBAUTCENF where DBCODRTA = ''OK'' AND DBVLRDEB > 0')"""
    # Ejecutar consulta y guardar resultado en DataFrame
    df4 = pd.read_sql(query3, conn)
    # Cerrar la conexión 
    conn.close()
    
    
    #Conexion a df6
    server = 'fabogcubox,1433'  # Nota: doble barra y puerto separado por coma
    database = 'Finandina_Cartera'
    driver = '{ODBC Driver 17 for SQL Server}'
    # Crear cadena de conexión
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    # Conectar a la base de datos 
    conn = pyodbc.connect(conn_str)
    # Consulta SQL ProductoActivo
    query4 = """SELECT Nid, [Celular1], [Celular2], [Email1], [Email2] FROM fabogcubox.[Finandina_cartera].dbo.[011 BaseSegmentacionDinamica 201911>]"""
    # Ejecutar consulta y guardar resultado en DataFrame
    df6 = pd.read_sql(query4, conn)
    # Cerrar la conexión 
    conn.close()
 
 
    #Conexion a df7
    server = 'fabogcubox,1433'  # Nota: doble barra y puerto separado por coma
    database = 'Finandina_Cartera'
    driver = '{ODBC Driver 17 for SQL Server}'
    # Crear cadena de conexión
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    # Conectar a la base de datos 
    conn = pyodbc.connect(conn_str)
    # Consulta SQL ProductoActivo
    query5 = """Select * from openquery (DB2400_182,'select CUNA3,CUSSNR from bnkprd01.cup003')"""
    # Ejecutar consulta y guardar resultado en DataFrame
    df7 = pd.read_sql(query5, conn)
    # Cerrar la conexión 
    conn.close()
    #Copia para df5
    df5=df2.copy()
    # Resultados df1
    print(df1.head())
    # Resultados df2 Y df5
    print(df2.head())
    # Resultados df3
    print(df3.head())
    # Resultados df4
    print(df4.head())
    # Resultados df5
    print(df5.head())
    # Resultados df6
    print(df6.head())
    # Resultados df7
    print(df7.head())
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    ## Transformacion
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    ## Carga
    #-------------------------------------------------------------------------------------------------------------------------------------------------
else:
    print(f"⏸️ Hoy ({hoy}) NO es el primer día hábil del mes ({primer_habil}). No se ejecuta nada.")


## extracion de datos

In [ ]:
# ========================
# CALCULO DE FECHAS SQL PARA EXTRAER EL PRIMER DIA DEL MES ANTERIOR Y ACTUAL
# ========================

inicio_mes_actual = hoy.replace(day=1)
inicio_mes_anterior = (inicio_mes_actual - timedelta(days=1)).replace(day=1)

fecha_inicio = inicio_mes_anterior.strftime('%Y-%m-%d')
fecha_fin = inicio_mes_actual.strftime('%Y-%m-%d')

fecha_inicio_yyyymmdd = inicio_mes_anterior.strftime('%Y%m%d')
fecha_fin_yyyymmdd = inicio_mes_actual.strftime('%Y%m%d')

# Conexión a df1
server = 'fabogcubox,1433'  # Nota: puerto separado por coma
database = 'Finandina_Cartera'
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Consulta SQL con fechas interpoladas
query = f"""
SELECT 
    C.Fechapago, C.HORPAG, C.MINPAG, C.TIPO_PRODUCTO,
    C.CFLNME AS Tipo_subproducto, C.CUX1AP, C.NUMDOC,
    C.CODSUC, C.NUMCRE, C.NOMCLI AS Nombre_Cliente,
    C.NUCREX, C.ORIPAG AS Origen_pago,
    CASE 
        WHEN ORIPAG = 1 THEN 'Conavi'
        WHEN ORIPAG = 2 THEN 'Banco caja social'
        WHEN ORIPAG = 3 THEN 'Banco de Bogotá'
        WHEN ORIPAG = 4 THEN 'Citibank'
        WHEN ORIPAG = 5 THEN 'Banco de occidente'
        WHEN ORIPAG = 6 THEN 'PSE'
        WHEN ORIPAG = 7 THEN 'Finazauto'
        WHEN ORIPAG = 8 THEN 'Bancolombia'
        WHEN ORIPAG = 10 THEN 'Banco de occidente ACH'
        WHEN ORIPAG = 11 THEN 'Banco AV Villas'
        WHEN ORIPAG = 12 THEN 'Cenit'
        WHEN ORIPAG = 13 THEN 'Banco Davivienda'
        WHEN ORIPAG = 14 THEN 'Cpv - Avvillas'
        WHEN ORIPAG = 15 THEN 'Libranza - Davivienda'
        WHEN ORIPAG = 16 THEN 'Sudameris'
        WHEN ORIPAG = 17 THEN 'Débito Automático'
        WHEN ORIPAG = 19 THEN 'Ws Bancolombia'
        WHEN ORIPAG IN (30, 31, 32) THEN 'Cajas Depositarias'
        WHEN ORIPAG = 63 THEN 'Cajas Finandina (Cheque TDC)'
        WHEN ORIPAG = 64 THEN 'Banca Línea - Finandina (TDC)'
        WHEN ORIPAG = 65 THEN 'Banco Finandina Pago TDC Dirigido'
        WHEN ORIPAG = 66 THEN 'Banco Finandina Pago Alterno TDC'
        ELSE NULL 
    END AS Descripcion_origen_pago,
    C.FORPAG AS Forma_pago, C.VLRPAG AS Valor_pagado,
    C.CIUREC, C.DESOBS, C.CODAPL
FROM (
    SELECT 
        CONCAT(
            SUBSTRING(B.FECPAG1, 1, 4), '-', 
            SUBSTRING(B.FECPAG1, 5, 2), '-', 
            SUBSTRING(B.FECPAG1, 7, 2)
        ) AS Fechapago, 
        B.*
    FROM (
        SELECT 
            CASE 
                WHEN a.CUX1AP IN (50, 51, 53, 33, 30) THEN 'CREDITO'
                WHEN a.CUX1AP = 20 THEN 'AHORRO'
                WHEN a.CUX1AP IS NULL AND LEN(a.NUCREX) IN (5, 6, 15, 16) THEN 'TDC'
                WHEN a.CUX1AP IS NULL AND LEFT(a.NUMCRE, 1) = '4' THEN 'TDC'
                WHEN a.CUX1AP IS NULL AND LEN(a.NUCREX) = 10 THEN 'CREDITO'
                WHEN LEN(a.NUMCRE) = 16 AND LEFT(a.NUMCRE, 1) = '4' THEN 'TDC'
                WHEN a.CUX1AP IS NULL AND LEN(a.NUMCRE) = 16 THEN 'CREDITO'
                ELSE 'NULL'
            END AS TIPO_PRODUCTO,
            CAST(FECPAG AS varchar) AS FECPAG1,
            A.*
        FROM OPENQUERY(DB2400_182, 
            'SELECT 
                IFNULL(c.numdoc, cc.CUSSNR) AS numdoc,
                CXP.CUX1AP,
                pcl.*,
                TP.CFLNME
            FROM bnkprd01.pagcncl pcl
            LEFT JOIN inttarcre.sattarjet t ON t.pan = pcl.nucrex
            LEFT JOIN inttarcre.SATBENEFI b ON b.cuenta = t.cuenta
            LEFT JOIN inttarcre.SATDACOPE c ON c.identcli = b.identcli
            LEFT JOIN BNKPRD01.LNP00301 CR ON CR.LNNOTE = pcl.nucrex
            LEFT JOIN BNKPRD01.CFP503 TP ON TP.CFTYP = CR.LNTYPE
            LEFT JOIN BNKPRD01.CUP009 CXP ON CR.LNNOTE = CXP.CUX1AC
            LEFT JOIN BNKPRD01.CUP00301 Cc ON Cc.CUNBR = CXP.CUX1CS
            WHERE pcl.fecpag >= {fecha_inicio_yyyymmdd} AND pcl.fecpag < {fecha_fin_yyyymmdd}
            ') A
    ) B
) C
WHERE C.Fechapago >= '{fecha_inicio}' AND C.Fechapago < '{fecha_fin}'
"""

conn = pyodbc.connect(conn_str)
df1 = pd.read_sql(query, conn)
conn.close()

In [ ]:
# Conexion a df2 (df2 es el mismo df5)
server = 'fabogriesgo\\riesgodb,1433'  # Nota: doble barra invertida para escapar la barra
database = 'Productos y transaccionalidad'
driver = '{ODBC Driver 17 for SQL Server}'

# Crear cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)

# Consulta SQL ProductoActivo
query1 = """SELECT * FROM [Productos y transaccionalidad].[dbo].[ProductoActivo]"""

# Ejecutar consulta y guardar resultado en DataFrame
df2 = pd.read_sql(query1, conn)

# Cerrar la conexión
conn.close()


In [ ]:
# Configuración de conexión
server = 'fabogsqlclu,1433'
database = 'PSE'
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)

# Consulta SQL: datos del año actual hasta hoy
query2 = """
SELECT * 
FROM [PSE].[dbo].[TransactionsInfo]
WHERE SoliciteDate >= DATEADD(MONTH, -1, DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1))
  AND SoliciteDate < DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1)
  AND LOWER(TransactionState) = 'ok'

"""

# Ejecutar la consulta
df3 = pd.read_sql(query2, conn)

# Cerrar conexión
conn.close()



In [168]:
#librerias estandar
import pyodbc  # Conexión a bases de datos mediante ODBC (SQL Server, Oracle, etc.)
import pandas as pd  # Manipulación y análisis de datos en estructuras como DataFrames (leer CSV, Excel, SQL, etc.)
import numpy as np  # Soporte para cálculos numéricos y manejo de arrays/matrices
import openpyxl  # Leer y escribir archivos Excel (.xlsx), útil para automatización de reportes
from openpyxl import load_workbook  # Cargar un archivo de Excel existente para modificarlo
from openpyxl.styles import NamedStyle  # Crear estilos personalizados para celdas de Excel (fuentes, bordes, etc.)
from openpyxl.utils import get_column_letter  # Convertir número de columna a letra (por ejemplo, 1 → 'A')
from datetime import date,datetime, timedelta # Trabajar con fechas y horas (obtener fecha actual, calcular diferencias, formatear fechas, etc.)
import os  # Interacción con el sistema operativo (rutas de archivos, crear carpetas, verificar existencia de archivos, etc.)
import holidays

In [169]:
#df1
# Ruta del archivo CSV
ruta_excel1 = r"C:\Users\davpae\Downloads\df1.csv"
# Leer el archivo CSV con codificación latin-1
df1 = pd.read_csv(ruta_excel1, encoding='latin1')

#df2
# Ruta del archivo CSV
ruta_excel2 = r"C:\Users\davpae\Downloads\df2.csv"
# Leer el archivo CSV con codificación latin-1
df2 = pd.read_csv(ruta_excel2, encoding='latin1')

##df3
# Ruta del archivo CSV
ruta_excel3 = r"C:\Users\davpae\Downloads\df3.csv"
# Leer el archivo CSV con codificación latin-1
df3 = pd.read_csv(ruta_excel3, encoding='latin1')

C:\Users\davpae\AppData\Local\Temp\ipykernel_18096\3348705013.py:17: DtypeWarning: Columns (10,11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv(ruta_excel3, encoding='latin1')


In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [170]:
# CAMBIAR A LONG INT64
df1['NUMDOC'] = pd.to_numeric(df1['NUMDOC'], errors='coerce').astype('Int64')  # Int64 permite nulos
df1['NUCREX'] = pd.to_numeric(df1['NUCREX'], errors='coerce').astype('Int64')

# Excluir de las filas de NUMDOC bajo el patrón 800149923
df8 = df1.copy()
df10 = df8.loc[df8['NUMDOC'].astype(str).str.contains('800149923')]
df9 = df1.loc[~df1['NUMDOC'].astype(str).str.contains('800149923')]

# Conversión de tipos en df2
df2['Obligacion'] = pd.to_numeric(df2['Obligacion'], errors='coerce').astype('Int64')
df2['DocumentoCliente'] = pd.to_numeric(df2['DocumentoCliente'], errors='coerce').astype('Int64')

# Merge left de quienes contienen 800149923 con df2
df11 = df10.merge(
    df2,
    how='left',
    left_on='NUCREX',
    right_on='Obligacion'
)

# Regla: si NUMDOC es igual a DocumentoCliente, se asigna 1
df11['NUMDOC'] = df11.apply(
    lambda row: 1 if row['NUMDOC'] == row['DocumentoCliente'] else row['NUMDOC'],
    axis=1
)

# Column filter
orden_columnas = [
    'Fechapago', 'HORPAG', 'MINPAG', 'TIPO_PRODUCTO', 'Tipo_subproducto', 'CUX1AP',
    'NUMDOC', 'CODSUC', 'NUMCRE', 'Nombre_Cliente', 'Origen_pago',
    'Descripcion_origen_pago', 'Forma_pago', 'Valor_pagado', 'CIUREC',
    'DESOBS', 'CODAPL'
]

# Filtrar columnas en df11
df11 = df11[orden_columnas]

# Concatenar df9 con df11 (sin hacer merge)
df12 = pd.concat([df9, df11], axis=0, ignore_index=True, sort=False)

# Transformar Fechapago a datetime (aaaa-mm-dd)
df12["Fechapago"] = pd.to_datetime(df12["Fechapago"].astype(str).str.strip(), format="%Y-%m-%d", errors="coerce").dt.date

# Redondeo de HORPAG y MINPAG a enteros
df12['HORPAG'] = df12['HORPAG'].round().astype('Int64')
df12['MINPAG'] = df12['MINPAG'].round().astype('Int64')


In [ ]:
df12

In [ ]:
df12.columns = df12.columns.str.strip()

In [171]:
# Asegurar que no haya valores nulos y convertir a string
# Filtrar filas donde el valor en TransactionState (en minúsculas) sea exactamente 'ok'
df3 = df3[df3['TransactionState'].str.contains("OK",case=False,na=False)]
# convertir a datetime
df3['SoliciteDate'] = pd.to_datetime(df3['SoliciteDate'], errors='coerce')
# Extraer hora y minuto
df3['Hour'] = df3['SoliciteDate'].dt.hour
df3['Minute'] = df3['SoliciteDate'].dt.minute
#Conversion a aaaa-mm-dd
df3['BankProcessDate'] = pd.to_datetime(df3['BankProcessDate'], errors='coerce').dt.date
df3['SoliciteDate'] = pd.to_datetime(df3['SoliciteDate'], errors='coerce').dt.date
df3['SoliciteDateINI'] = pd.to_datetime(df3['SoliciteDateINI'], errors='coerce').dt.date

In [ ]:
df3

In [ ]:
df3 = df3.rename(columns={
    'Reference3': 'NUCREX',
    'Reference2': 'NUMDOC',
    'SoliciteDate': 'Fechapago',
    'AmountValue': 'Valor_pagado', 
    'Hour': 'HORPAG',
    'Minute': 'MINPAG'
})


In [ ]:
print("=== Tipos en df12 ===")
print(df12[['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG']].dtypes)

print("\n=== Tipos en df3 ===")
print(df3[['Reference3', 'Reference2', 'SoliciteDate', 'AmountValue', 'Hour', 'Minute']].dtypes)


In [172]:
# --- LIMPIEZA GENERAL DE COLUMNAS: nombres sin espacios ---
df12.columns = df12.columns.str.strip()
df3.columns = df3.columns.str.strip()

# --- LIMPIEZA df12 ---
df12['NUCREX'] = df12['NUCREX'].astype(str).str.strip()
df12['NUMDOC'] = df12['NUMDOC'].astype(str).str.strip()
df12['Fechapago'] = pd.to_datetime(df12['Fechapago'].astype(str).str.strip(), errors='coerce').dt.strftime('%Y-%m-%d')
df12['Valor_pagado'] = pd.to_numeric(df12['Valor_pagado'].astype(str).str.strip(), errors='coerce').round(2)
df12['HORPAG'] = pd.to_numeric(df12['HORPAG'].astype(str).str.strip(), errors='coerce').astype('Int64')
df12['MINPAG'] = pd.to_numeric(df12['MINPAG'].astype(str).str.strip(), errors='coerce').astype('Int64')

# --- LIMPIEZA df3 ---
df3['Reference3'] = df3['Reference3'].astype(str).str.strip()
df3['Reference2'] = df3['Reference2'].astype(str).str.strip()
df3['SoliciteDate'] = pd.to_datetime(df3['SoliciteDate'].astype(str).str.strip(), errors='coerce').dt.strftime('%Y-%m-%d')
df3['AmountValue'] = pd.to_numeric(df3['AmountValue'].astype(str).str.strip(), errors='coerce').round(2)
df3['Hour'] = pd.to_numeric(df3['Hour'].astype(str).str.strip(), errors='coerce').astype('Int64')
df3['Minute'] = pd.to_numeric(df3['Minute'].astype(str).str.strip(), errors='coerce').astype('Int64')

# Redondear monto a 0 decimales
df12['Valor_pagado'] = pd.to_numeric(df12['Valor_pagado'], errors='coerce').round(0)
df3['AmountValue'] = pd.to_numeric(df3['AmountValue'], errors='coerce').round(0)

# Redondear horas y minutos al entero más cercano
df12['HORPAG'] = pd.to_numeric(df12['HORPAG'], errors='coerce').round(0).astype('Int64')
df12['MINPAG'] = pd.to_numeric(df12['MINPAG'], errors='coerce').round(0).astype('Int64')
df3['Hour'] = pd.to_numeric(df3['Hour'], errors='coerce').round(0).astype('Int64')
df3['Minute'] = pd.to_numeric(df3['Minute'], errors='coerce').round(0).astype('Int64')






In [ ]:
# Convertir todos los campos a string antes de concatenar
df12['LLAVE'] = (
    df12['NUCREX'].astype(str) + '|' +
    df12['NUMDOC'].astype(str) + '|' +
    df12['Fechapago'].astype(str) + '|' +
    df12['Valor_pagado'].astype(str) + '|' +
    df12['HORPAG'].astype(str) + '|' +
    df12['MINPAG'].astype(str)
)

df3['LLAVE'] = (
    df3['Reference3'].astype(str) + '|' +
    df3['Reference2'].astype(str) + '|' +
    df3['SoliciteDate'].astype(str) + '|' +
    df3['AmountValue'].astype(str) + '|' +
    df3['Hour'].astype(str) + '|' +
    df3['Minute'].astype(str)
)

In [ ]:
df3 = df3.drop_duplicates(subset='LLAVE', keep='last')

In [ ]:
df3

In [ ]:
columnas = ['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG']

print("Tipos de datos en df12:")
print(df12[columnas].dtypes)
#print("\nValores en df12:")
#print(df12[columnas].head())

print("\nTipos de datos en df3:")
print(df3[columnas].dtypes)
#print("\nValores en df3:")
#print(df3[columnas].head())

In [ ]:
df12_pse = df12[df12['Descripcion_origen_pago'].astype(str).str.strip().str.upper() == 'PSE'].copy()
df_merge_exacto = df12_pse.merge(
    df3,
    how='left',
    left_on=['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG'],
    right_on=['Reference3', 'Reference2', 'SoliciteDate', 'AmountValue', 'Hour', 'Minute'],
    indicator=True
)
df_matched = df_merge_exacto[df_merge_exacto['_merge'] == 'both']
print("Total matches exactos:", len(df_matched))  # Aquí deberías ver 102.749
df12_pse_matched = df_matched.copy()
df12_pse_matched['CIUREC'] = df12_pse_matched['BankName']


In [173]:
#df13 = df12.merge(
##    df3,
#    how='left',
#    left_on='LLAVE',
##    right_on='LLAVE'
#)
#df13 = df12.merge(df3, on=['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG'], how='left')


# Merge LEFT de df12 con df3 usando las columnas correspondientes
df13 = df12.merge(
    df3,
    how='left',
    left_on=['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG'],
    right_on=['Reference3', 'Reference2', 'SoliciteDate', 'AmountValue', 'Hour', 'Minute'],
    indicator=True
)


In [ ]:
df13


In [ ]:
#df13 = df13.drop_duplicates(
#    subset=['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG'],
#    keep='last'
#)


In [174]:
# Definir el orden de las columnas deseadas
orden_columnas = ['Fechapago', 'HORPAG', 'MINPAG', 'TIPO_PRODUCTO', 'Tipo_subproducto', 'CUX1AP', 'NUMDOC', 'CODSUC', 'NUMCRE', 'Nombre_Cliente', 'NUCREX', 'Origen_pago', 'Descripcion_origen_pago', 'Forma_pago', 'Valor_pagado', 'CIUREC', 'DESOBS', 'CODAPL', 'BankName','Hour','Minute']
# Filtrar y organizar el DataFrame según el orden de columnas
df13= df13[orden_columnas]

In [ ]:
df13

In [ ]:
df13['CIUREC'] = np.where(
    df13['Descripcion_origen_pago'].astype(str).str.strip().str.upper() == 'PSE',
    df13['BankName'],
    np.nan  # o '', si prefieres cadena vacía
)

In [177]:
df13.to_excel(r'C:\Users\davpae\Downloads\df13.xlsx', index=False)

In [ ]:
df12_pse = df12[df12['Descripcion_origen_pago'].astype(str).str.strip().str.upper() == 'PSE']

df_pse_merge = df12_pse.merge(
    df3,
    how='left',
    left_on=['NUCREX', 'NUMDOC', 'Fechapago', 'Valor_pagado', 'HORPAG', 'MINPAG'],
    right_on=['Reference3', 'Reference2', 'SoliciteDate', 'AmountValue', 'Hour', 'Minute'],
    indicator=True
)

In [ ]:
print(df_pse_merge['_merge'].value_counts())

In [ ]:
# Ruta del archivo Excel Leads
ruta_excelp1 = r"C:\Users\davpae\Downloads\preca2.xlsx"
# Leer el archivo Excel
dfp1= pd.read_excel(ruta_excelp1)

In [ ]:
def reporte_merge_progresivo(df12, df3):
    pasos = [
        (['NUCREX', 'NUMDOC'], ['Reference3', 'Reference2']),
        (['Fechapago'], ['SoliciteDate']),
        (['Valor_pagado'], ['AmountValue']),
        (['HORPAG'], ['Hour']),
        (['MINPAG'], ['Minute'])
    ]

    left_keys = []
    right_keys = []
    df_temp = df12.copy()

    print("🔁 INICIO DEL REPORTE PROGRESIVO DE MERGE")
    print("Total base df12:", len(df12))

    for i, (lk, rk) in enumerate(pasos, start=1):
        left_keys += lk
        right_keys += rk

        df_merged = df_temp.merge(df3, how='left', left_on=left_keys, right_on=right_keys, indicator=True)
        count_both = (df_merged['_merge'] == 'both').sum()

        print(f"Paso {i}: Merge en {left_keys} vs {right_keys}")
        print(f"➡️ Registros que cruzan: {count_both}")
        print("-" * 40)

# Llama a esta función con tus DataFrames ya limpiados
# reporte_merge_progresivo(df12, df3)


In [ ]:
reporte_merge_progresivo(df12, df3)

In [ ]:
# Ruta del archivo CSV
ruta_excelp2 = r"C:\Users\davpae\Downloads\preca1.csv"

# Leer el archivo CSV con codificación latin-1
dfp2 = pd.read_csv(ruta_excelp2, encoding='latin1')